In [1]:
# 농수축 & 이미용 EDAddd
# !pip install plotly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import requests
import xmltodict
import re

import pickle
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# pd.set_option('display.max_rows', 100)
%matplotlib inline
plt.style.use('ggplot') # 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
sum_df = pd.read_csv('c:/users/moon/downloads/sum_df.csv')
del sum_df['Unnamed: 0']
sum_df.head()

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,...,grade,season,timeslot,holiday,sales_group,52_week,brand_name,brand_power,mask_ratio,parcel_ratio
0,2019-01-01 00:00:00,60.0,100377,201225,그렉노먼 남성 캐시터치 터틀넥 4종,의류,69900,60105000.0,858.0,2019,...,3,4,7,1,2.0,1,그렉노먼,21.052242,0.04095,0.52582
1,2019-01-01 01:00:00,57.1,100193,200652,레이프릴 무빙 맥시풀커버 브라팬티,속옷,69900,43858000.0,627.0,2019,...,4,4,7,1,1.0,1,레이프릴,29.260575,0.04095,0.52582
2,2019-01-01 06:00:00,60.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,32696000.0,820.0,2019,...,4,4,1,1,2.0,1,테이트,16.386845,0.04095,0.52582
3,2019-01-01 07:00:00,60.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,47878000.0,812.0,2019,...,4,4,1,1,2.0,1,오모떼,18.293730,0.04095,0.52582
4,2019-01-01 08:00:00,60.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,99736000.0,1665.0,2019,...,3,4,1,1,4.0,1,CERINI,29.312362,0.04095,0.52582


In [3]:
def df_split(group):
    return sum_df[sum_df['p_group']==group]

def sales_split(group):
    return pd.qcut(sum_df[sum_df['p_group']==group].sales, q=5,labels=[1,2,3,4,5]).to_frame().rename(columns={'sales':'sales_group'})

def unit_split(group):
    return pd.qcut(sum_df[sum_df['p_group']==group].unit_price, q=5,labels=[1,2,3,4,5]).to_frame().rename(columns={'unit_price':'unit_group'})

# interested_group_list = list(sum_df.p_group.unique())
# # ['의류', '속옷', '주방', '농수축', '이미용', '가전', '생활용품', '건강기능', '잡화', '가구', '침구']
# for group in interested_group_list:
#     for idx, value in zip(sales_split(group).index,sales_split(group).sales_group):
#         sum_df.loc[idx,'sales_group'] = value

In [4]:
# # sum_df = sum_df[(sum_df['sales_group'] >= 4)|(sum_df['sales_group'] <= 2)].reset_index(drop=True)
sum_df['score'] = 'temp'
for row, value in enumerate(sum_df.sales_group):
    if value <= 2:
        sum_df.score[row] = 0
    elif value >= 4:
        sum_df.score[row] = 1

C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
interested_group = list(sum_df.p_group.unique())

group_solution_dict = {}

for group in interested_group:
    df = sum_df[sum_df['p_group']==group]
    solution_list = []
    df_total_sales = df.sales.sum()
    print(group,'의 unit_price 개수:',len(df.unit_price.unique()))
    for idx, x in enumerate(sorted(df.unit_price.unique())):
        if idx % 10 == 0:
            print(idx, x)
        for y in sorted(df.unit_price.unique()):
            if x >= y:
                continue
            elif ((df[(df['unit_price'] >= x) & (df['unit_price'] <= y)].sales.sum() / df_total_sales) >= 0.495) &\
            ((df[(df['unit_price'] >= x) & (df['unit_price'] <= y)].sales.sum() / df_total_sales) <= 0.505):
                solution_list.append((x,y))
    group_solution_dict[group]=solution_list
group_solution_dict

의류 의 unit_price 개수: 48
0 29000
10 44900
20 66000
30 129000
40 399000
속옷 의 unit_price 개수: 26
0 29900
10 63200
20 109000
주방 의 unit_price 개수: 75
0 24900
10 50900
20 64800
30 79800
40 93000
50 123000
60 179000
70 254000
농수축 의 unit_price 개수: 34
0 28900
10 41900
20 55900
30 73900
이미용 의 unit_price 개수: 14
0 39800
10 69800
가전 의 unit_price 개수: 33
0 164000
10 829000
20 1330000
30 1959000
생활용품 의 unit_price 개수: 61
0 12800
10 39900
20 52900
30 89000
40 129000
50 219000
60 1880000
건강기능 의 unit_price 개수: 26
0 39900
10 69900
20 129000
잡화 의 unit_price 개수: 50
0 29000
10 49800
20 79800
30 169000
40 380000
가구 의 unit_price 개수: 18
0 89000
10 799000
침구 의 unit_price 개수: 9
0 39900


{'의류': [(29000, 49800),
  (34000, 59000),
  (40800, 79000),
  (44000, 79000),
  (49800, 299000),
  (49800, 399000),
  (49800, 499000),
  (49800, 690000),
  (49900, 399000),
  (49900, 499000),
  (49900, 690000),
  (49900, 699000),
  (49900, 1490000),
  (49900, 1590000),
  (49900, 1690000),
  (49900, 2490000)],
 '속옷': [(59900, 109000)],
 '주방': [(24900, 89000),
  (29800, 89000),
  (39900, 89800),
  (39900, 90000),
  (39900, 90800),
  (39900, 93000),
  (41900, 99000),
  (41900, 104000),
  (49000, 99000),
  (49000, 104000),
  (49800, 104000),
  (61800, 154000),
  (79800, 198000),
  (79900, 198000),
  (79900, 199000),
  (80000, 198000),
  (80000, 199000),
  (80000, 208000),
  (84000, 198000),
  (84000, 199000),
  (84000, 208000),
  (84800, 208000),
  (88800, 209000),
  (88800, 219000),
  (88800, 228000),
  (88800, 229000),
  (89000, 209000),
  (89000, 219000),
  (89000, 228000),
  (89000, 229000),
  (89000, 238000),
  (89000, 239000),
  (89000, 254000),
  (89000, 259000),
  (89000, 309000),


In [6]:
picked_solutions = {}

for group in interested_group:
    if len(group_solution_dict[group]) == 0:
        continue
    elif len(group_solution_dict[group]) == 1:
        picked_solutions[group] = group_solution_dict[group][0]
        continue
    else:
        min_range = abs(group_solution_dict[group][0][0]-group_solution_dict[group][0][1])
        chosen_values = (group_solution_dict[group][0][0], group_solution_dict[group][0][1])
        for x, y in group_solution_dict[group][1:]:
            if min_range > abs(x-y):
                min_range = abs(x-y)
                chosen_values = (x,y)
    picked_solutions[group] = chosen_values
    
picked_solutions

{'의류': (29000, 49800),
 '속옷': (59900, 109000),
 '주방': (39900, 89800),
 '농수축': (40900, 60900),
 '생활용품': (29800, 59800),
 '건강기능': (59000, 69300),
 '잡화': (29800, 48900)}

In [7]:
rest = ['이미용', '가전', '가구', '침구']
group_solution_dict = {}

for group in rest:
    df = sum_df[sum_df['p_group']==group]
    solution_list = []
    df_total_sales = df.sales.sum()
    
    if group == '이미용':
        upper = 0.55
        lower = 0.45
    elif group == '가전':
        upper = 0.51
        lower = 0.49
    elif group == '가구':
        upper = 0.56
        lower = 0.44
    elif group == '침구':
        upper = 0.59
        lower = 0.44
    
    for idx, x in enumerate(sorted(df.unit_price.unique())):
        for y in sorted(df.unit_price.unique()):
            if x >= y:
                continue
            elif ((df[(df['unit_price'] >= x) & (df['unit_price'] <= y)].sales.sum() / df_total_sales) >= lower) &\
            ((df[(df['unit_price'] >= x) & (df['unit_price'] <= y)].sales.sum() / df_total_sales) <= upper):
                solution_list.append((x,y))
    group_solution_dict[group]=solution_list
group_solution_dict

{'이미용': [(39900, 79000), (39900, 79900)],
 '가전': [(509000, 1409000), (509000, 1419000)],
 '가구': [(548000, 589000)],
 '침구': [(39900, 49900)]}

In [8]:
picked_solutions_rest = {}

for group in rest:
    if len(group_solution_dict[group]) == 0:
        continue
    elif len(group_solution_dict[group]) == 1:
        picked_solutions_rest[group] = group_solution_dict[group][0]
        continue
    else:
        min_range = abs(group_solution_dict[group][0][0]-group_solution_dict[group][0][1])
        chosen_values = (group_solution_dict[group][0][0], group_solution_dict[group][0][1])
        for x, y in group_solution_dict[group][1:]:
            if min_range > abs(x-y):
                min_range = abs(x-y)
                chosen_values = (x,y)
    picked_solutions_rest[group] = chosen_values
    
picked_solutions_rest

{'이미용': (39900, 79000),
 '가전': (509000, 1409000),
 '가구': (548000, 589000),
 '침구': (39900, 49900)}

In [9]:
final_solutions = dict(picked_solutions_rest.items( ) | picked_solutions.items( ) ) 

In [10]:
high_low_dict = {}
for key, value in list(final_solutions.items()):
    x, y = value
    low_df = df_split(key)[(df_split(key).unit_price >= x) & (df_split(key).unit_price <= y)&((df_split(key).sales_group==1)|(df_split(key).sales_group==2))]
    high_df = df_split(key)[(df_split(key).unit_price >= x) & (df_split(key).unit_price <= y)&((df_split(key).sales_group==4)|(df_split(key).sales_group==5))]
    high_low_dict[key] = pd.concat([high_df,low_df], axis=0).reset_index(drop=True)

In [12]:
# with open('c:/bc/high_low_dict.pickle', 'wb') as f:
#     pickle.dump(high_low_dict, f)

In [13]:
high_low_dict['가전']

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,...,season,timeslot,holiday,sales_group,52_week,brand_name,brand_power,mask_ratio,parcel_ratio,score
0,2019-01-15 23:00:00,60.0,100577,201758,일시불 삼성 노트북 9 메탈 기본형 NT900X5J-K14,가전,999000,231475000.0,181.0,2019,...,4,6,0,4.0,3,삼성,2.135714,0.41285,1.53368,1
1,2019-02-07 22:00:00,60.0,100577,201758,일시불 삼성 노트북 9 메탈 기본형 NT900X5J-K14,가전,999000,200420000.0,159.0,2019,...,4,6,0,4.0,6,삼성,2.135714,0.04150,1.32119,1
2,2019-02-08 20:00:00,60.0,100575,201737,일시불 삼성 UHD TV UN55NU7010FXKR,가전,1000000,202516000.0,157.0,2019,...,4,5,0,4.0,6,삼성,2.135714,0.03696,7.03844,1
3,2019-02-09 21:00:00,60.0,100577,201758,일시불 삼성 노트북 9 메탈 기본형 NT900X5J-K14,가전,999000,224372000.0,179.0,2019,...,4,6,1,4.0,6,삼성,2.135714,0.03048,1.70138,1
4,2019-02-13 21:00:00,60.0,100155,200554,일시불 LG전자 매직스페이스 냉장고,가전,1409000,239385000.0,164.0,2019,...,4,6,0,4.0,7,LG,3.102464,0.04149,1.76847,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2019-12-03 23:20:00,60.0,100575,201778,초특가 무이자 삼성 UHD TVUN75RU7150FXKR,가전,1050000,77156000.0,46.0,2019,...,4,6,0,1.0,49,삼성,2.135714,0.06899,1.53750,0
266,2019-12-05 21:20:00,60.0,100148,200411,LG 울트라HD TV 55UM7900BNA,가전,1200000,174174000.0,108.0,2019,...,4,6,0,2.0,49,LG,3.102464,0.06877,1.29030,0
267,2019-12-10 22:20:00,60.0,100575,201778,초특가 무이자 삼성 UHD TVUN75RU7150FXKR,가전,1050000,156533000.0,97.0,2019,...,4,6,0,2.0,50,삼성,2.135714,0.12329,1.37896,0
268,2019-12-14 21:20:00,60.0,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1359000,155097000.0,110.0,2019,...,4,6,1,2.0,50,LG,3.102464,0.04353,0.72301,0
